In [1]:
import os
import random
import json
from glob import glob
from pathlib import Path
import yaml
from tqdm import tqdm
import IPython.display as ipd

import numpy as np
import pandas as pd

import matplotlib as mpl
cmap = mpl.cm.get_cmap('coolwarm')
import matplotlib.pyplot as plt

import librosa
import wandb
from sklearn import metrics

import torch
from torch import nn
from torch.nn import functional as F
from torch.cuda.amp import autocast, GradScaler

from timm.scheduler import CosineLRScheduler

In [2]:
from config import CFG
from pytorch_model import BirdCLEF23Net
from pytorch_wav2logmel import Wav2Logmel
import pytorch_modeler as modeler
import pytorch_preprocessing as prep
import common as com

In [3]:
modeler.set_seed(CFG.seed)
# setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print('Debug :', CFG.debug)

In [4]:
# Try to get the API key from Kaggle secrets
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    # Login to wandb with the API key
    wandb.login(key=api_key)
    print('kaggle notebook mode')
except:
    key_path = '/kaggle/input/wandb_key.txt'
    p = Path(key_path)
    api_key = p.read_text()
    wandb.login(key=api_key)
    print('local mode')

In [5]:
from datetime import datetime
import pytz

# 日本時間のタイムゾーンを設定
jst = pytz.timezone('Asia/Tokyo')

# 現在時刻を取得し、日本時間に変換
now = datetime.now(jst)

# 現在時刻を文字列に変換
now_str = now.strftime('%Y-%m-%d %H:%M:%S')

print(now_str)

In [6]:
import yaml
from tqdm import tqdm

def wandb_init(fold):
    config = {k: v for k, v in dict(vars(CFG)).items() if '__' not in k}
    config.update({"fold": int(fold)})
    yaml.dump(config, open(f'./config fold-{fold}.yaml', 'w'), )
    config = yaml.load(open(f'./config fold-{fold}.yaml', 'r'), Loader=yaml.FullLoader)
    run = wandb.init(project="birdclef-2023-public",
                     name=f"fold-{fold}|dim-{CFG.img_size[1]}x{CFG.img_size[0]}|model-{CFG.model_name}|{now_str}",
                     config=config,
                     group=CFG.comment,
                     save_code=True, )
    return run


def log_wandb(valid_df):
    save_df = valid_df.query("miss==True")
    save_df.loc[:, 'pred_name'] = save_df.pred.map(CFG.label2name)
    save_df.loc[:, 'target_name'] = save_df.target.map(CFG.label2name)
    if CFG.debug:
        save_df = save_df.iloc[:CFG.batch_size * CFG.valid_bs]
    noimg_cols = [*CFG.tab_cols, 'target', 'pred', 'target_name', 'pred_name']
    save_df = save_df.loc[:, noimg_cols]

    data = []
    for idx, row in tqdm(save_df.iterrows(), total=len(save_df), desc='wandb ', position=0, leave=True):
        filepath = '/kaggle/input/birdclef-2023/train_audio/' + row.filename
        audio, sr = librosa.load(filepath, sr=None)
        data += [[*row.tolist(), wandb.Audio(audio, caption=row.filename, sample_rate=sr)]]
    wandb_table = wandb.Table(data=data, columns=[*noimg_cols, 'audio'])
    wandb.log({'best': scores,
               'table': wandb_table,
               })

In [7]:
df = pd.read_csv(f'{CFG.BASE_PATH}/train_metadata.csv')
filename = df['filename'].str.replace('.ogg', '.wav')
df['filepath'] = CFG.BASE_PATH + '/train_audio_wav/' + filename
df['target'] = df.primary_label.map(CFG.name2label)
df['birdclef'] = '23'
#df['filename'] = df.filepath.map(lambda x: x.split('/')[-1])
df['xc_id'] = df.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
df.head(2)

  primary_label secondary_labels      type  latitude  longitude  \
0       abethr1               []  ['song']    4.3906    38.2788   
1       abethr1               []  ['call']   -2.9524    38.2921   

      scientific_name               common_name         author  \
0  Turdus tephronotus  African Bare-eyed Thrush  Rolf A. de By   
1  Turdus tephronotus  African Bare-eyed Thrush  James Bradley   

                                             license  rating  \
0  Creative Commons Attribution-NonCommercial-Sha...     4.0   
1  Creative Commons Attribution-NonCommercial-Sha...     3.5   

                                 url              filename  \
0  https://www.xeno-canto.org/128013  abethr1/XC128013.ogg   
1  https://www.xeno-canto.org/363501  abethr1/XC363501.ogg   

                                            filepath  target birdclef  \
0  /kaggle/input/birdclef-2023/train_audio_wav/ab...       0       23   
1  /kaggle/input/birdclef-2023/train_audio_wav/ab...       0       23   


In [8]:
# BirdCLEF-2020
df_20 = pd.read_csv(f'{CFG.BASE_PATH_20}/train.csv')
df_20['primary_label'] = df_20['ebird_code']
df_20['filepath'] = CFG.BASE_PATH_20 + '/train_audio/' + df_20.primary_label + '/' + df_20.filename
df_20['scientific_name'] = df_20['sci_name']
df_20['common_name'] = df_20['species']
df_20['target'] = df_20.primary_label.map(CFG.name2label_pre)
df_20['birdclef'] = '20'
#assert tf.io.gfile.exists(df_20.filepath.iloc[0])

# Xeno-Canto Extend by @vopani
df_xam = pd.read_csv(f'{CFG.BASE_PATH_xam}/train_extended.csv')
df_xam['filepath'] = CFG.BASE_PATH_xam + '/A-M/' + df_xam.ebird_code + '/' + df_xam.filename
df_xnz = pd.read_csv(f'{CFG.BASE_PATH_xnz}/train_extended.csv')
df_xnz['filepath'] = CFG.BASE_PATH_xnz + '/N-Z/' + df_xnz.ebird_code + '/' + df_xnz.filename
df_xc = pd.concat([df_xam, df_xnz], axis=0, ignore_index=True)
df_xc['primary_label'] = df_xc['ebird_code']
df_xc['scientific_name'] = df_xc['sci_name']
df_xc['common_name'] = df_xc['species']
df_xc['target'] = df_xc.primary_label.map(CFG.name2label_pre)
df_xc['birdclef'] = 'xc'
#assert tf.io.gfile.exists(df_xc.filepath.iloc[0])

# BirdCLEF-2021
df_21 = pd.read_csv(f'{CFG.BASE_PATH_21}/train_metadata.csv')
df_21['filepath'] = CFG.BASE_PATH_21 + '/train_short_audio/' + df_21.primary_label + '/' + df_21.filename
df_21['target'] = df_21.primary_label.map(CFG.name2label_pre)
df_21['birdclef'] = '21'
corrupt_paths = [f'{CFG.BASE_PATH_21}/train_short_audio/houwre/XC590621.ogg',
                 f'{CFG.BASE_PATH_21}/train_short_audio/cogdov/XC579430.ogg']
df_21 = df_21[~df_21.filepath.isin(corrupt_paths)] # remove all zero audios
#assert tf.io.gfile.exists(df_21.filepath.iloc[0])

# BirdCLEF-2022
df_22 = pd.read_csv(f'{CFG.BASE_PATH_22}/train_metadata.csv')
df_22['filepath'] = CFG.BASE_PATH_22 + '/train_audio/' + df_22.filename
df_22['target'] = df_22.primary_label.map(CFG.name2label_pre)
df_22['birdclef'] = '22'
#assert tf.io.gfile.exists(df_22.filepath.iloc[0])

# Merge 2021 and 2022 for pretraining
df_pre = pd.concat([df_20, df_21, df_22, df_xc], axis=0, ignore_index=True)
df_pre['filename'] = df_pre.filepath.map(lambda x: x.split('/')[-1])
df_pre['xc_id'] = df_pre.filepath.map(lambda x: x.split('/')[-1].split('.')[0])
nodup_idx = df_pre[['xc_id','primary_label','author']].drop_duplicates().index
df_pre = df_pre.loc[nodup_idx].reset_index(drop=True)

# # Remove duplicates
df_pre = df_pre[~df_pre.xc_id.isin(df.xc_id)].reset_index(drop=True)
corrupt_mp3s = json.load(open('/kaggle/input/birdclef-2023-dataset/corrupt_mp3_files.json','r'))
corrupt_mp3s = [path.replace('/kaggle/input','/kaggle/input/birdclef-2023-dataset') for path in corrupt_mp3s]
df_pre = df_pre[~df_pre.filepath.isin(corrupt_mp3s)]
df_pre = df_pre[['filename','filepath','primary_label','secondary_labels',
                 'rating','author','file_type','xc_id','scientific_name',
                'common_name','target','birdclef','bird_seen']]

# rename
df_pre['filepath']=df_pre['filepath'].str.replace('train_audio', 'train_audio_mono_wav')
df_pre['filepath']=df_pre['filepath'].str.replace('train_short_audio', 'train_short_audio_mono_wav')
df_pre['filepath']=df_pre['filepath'].str.replace('A-M', 'A-M_wav')
df_pre['filepath']=df_pre['filepath'].str.replace('mp3', 'wav')
df_pre['filepath']=df_pre['filepath'].str.replace('ogg', 'wav')
# fillna
df_pre=df_pre.fillna('')
# to 23 class_names
df_pre = df_pre[df_pre['primary_label'].isin(CFG.class_names)]
df_pre = df_pre.reset_index(drop=True)
df_pre['target'] = df_pre.primary_label.map(CFG.name2label)
df_pre['secondary_labels'] = '[]'

# Display rows
print("# Samples for Pre-Training: {:,}".format(len(df_pre)))
df_pre.head(2).style.set_caption("Pre-Training Data").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '16px')
    ]
}])

# Show distribution
plt.figure(figsize=(8, 4))
df_pre.birdclef.value_counts().plot.bar(color=[cmap(0.0),cmap(0.25), cmap(0.65), cmap(0.9)])
plt.xlabel("Dataset")
plt.ylabel("Count")
plt.title("Dataset distribution for Pre-Training")
plt.show()

In [9]:
# Import required packages
from sklearn.model_selection import StratifiedKFold

# Initialize the StratifiedKFold object with 5 splits and shuffle the data
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=CFG.seed)

# Reset the index of the dataframe
df = df.reset_index(drop=True)

# Create a new column in the dataframe to store the fold number for each row
df["fold"] = -1

# Iterate over the folds and assign the corresponding fold number to each row in the dataframe
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['primary_label'])):
    df.loc[val_idx, 'fold'] = fold

In [10]:
df_pre

         filename                                           filepath  \
0    XC139728.mp3  /kaggle/input/birdclef-2023-dataset/birdsong-r...   
1    XC142466.mp3  /kaggle/input/birdclef-2023-dataset/birdsong-r...   
2    XC142632.mp3  /kaggle/input/birdclef-2023-dataset/birdsong-r...   
3    XC142731.mp3  /kaggle/input/birdclef-2023-dataset/birdsong-r...   
4    XC142977.mp3  /kaggle/input/birdclef-2023-dataset/birdsong-r...   
..            ...                                                ...   
583  XC316378.mp3  /kaggle/input/birdclef-2023-dataset/xeno-canto...   
584  XC561783.mp3  /kaggle/input/birdclef-2023-dataset/xeno-canto...   
585  XC561777.mp3  /kaggle/input/birdclef-2023-dataset/xeno-canto...   
586  XC548833.mp3  /kaggle/input/birdclef-2023-dataset/xeno-canto...   
587  XC393118.mp3  /kaggle/input/birdclef-2023-dataset/xeno-canto...   

    primary_label secondary_labels  rating             author file_type  \
0          barswa               []     5.0        Paul Drive

In [11]:
import gc
def gc_collect():
    gc.collect()
    torch.cuda.empty_cache()

In [12]:
def drop_weight(weight):
    target_string='model.classifier'
    drop_keys = [key for key in weight.keys() if target_string in key]
    target_string='att_block'
    drop_keys += [key for key in weight.keys() if target_string in key]
    for key in drop_keys:
        if key in weight:
            del weight[key]
    return weight

# def name2onehot(multi_label_names: torch.Tensor):
#     multi_label_names = [ast.literal_eval(multi_label_name) for multi_label_name in multi_label_names]
#     multi_label_onehot=mlb.transform(multi_label_names)
#     multi_label_onehot=torch.tensor(multi_label_onehot).float()
#     return multi_label_onehot

In [13]:
oof_pred = []; oof_true = []; oof_val = []; oof_ids = []; oof_folds = [] 

num_classes = CFG.num_classes
df = df.copy()
for fold in range(CFG.num_fold):
    scaler = torch.cuda.amp.GradScaler()
    # Check if the fold is selected
    if fold not in CFG.selected_folds:
        continue
    
    # Initialize Weights and Biases
    if CFG.wandb:
        run = wandb_init(fold)
    
    # Compute batch size and number of samples to drop
    infer_bs = CFG.valid_bs
    drop_remainder = CFG.drop_remainder
    
    # Split dataset with cv filter
    if CFG.cv_filter:
        df = com.filter_data(df, thr=5)
        train_df = df.query("fold!=@fold | ~cv").reset_index(drop=True)
        valid_df = df.query("fold==@fold & cv").reset_index(drop=True)
    else:
        train_df = df.query("fold!=@fold").reset_index(drop=True)
        valid_df = df.query("fold==@fold").reset_index(drop=True)
    
    # Upsample train data
    train_df = com.upsample_data(train_df, thr=CFG.upsample_thr)
#     train_df = downsample_data(train_df, thr=500)

    # add additional data + fillna
    train_df = pd.concat([train_df, df_pre], axis=0).reset_index(drop=True)
    train_df['type'] = train_df['type'].fillna('[]')
    train_df['latitude'] = train_df['latitude'].fillna(0)
    train_df['longitude'] = train_df['longitude'].fillna(0)
    train_df['license'] = train_df['license'].fillna('')
    train_df['url'] = train_df['url'].fillna('')
    train_df['fold'] = train_df['fold'].fillna(99)
    train_df['cv'] = train_df['cv'].fillna(True)
    train_df['file_type'] = train_df['file_type'].fillna('wav')
    train_df['bird_seen'] = train_df['bird_seen'].fillna('')
    # Get file paths and labels
    train_paths = train_df.filepath.values; train_labels = train_df.target.values
    valid_paths = valid_df.filepath.values; valid_labels = valid_df.target.values

    # Shuffle the file paths and labels
    index = np.arange(len(train_paths))
    np.random.shuffle(index)
    train_paths  = train_paths[index]
    train_labels = train_labels[index]

    # wav
    train_ftype = list(map(lambda x: '.wav' in x, train_paths))
    valid_ftype = list(map(lambda x: '.wav' in x, valid_paths))

    # Compute the number of training and validation samples
    num_train = len(train_paths); num_valid = len(valid_paths)
        
    # Log the number of training and validation samples if Weights and Biases is being used
    if CFG.wandb:
        wandb.log({'num_train':num_train,
                   'num_valid':num_valid})
        
    # Build the training and validation datasets
    # For debugging
    if CFG.debug:
        min_samples = CFG.batch_size
        train_ds = prep.BirdDataset(train_df.iloc[:min_samples], is_train=True)
        valid_ds = prep.BirdDataset(valid_df, is_train=False)
    else:
        train_ds = prep.BirdDataset(train_df, is_train=True)
        valid_ds = prep.BirdDataset(valid_df, is_train=False)
    # dataloader
    train_dataloader, val_dataloader = modeler.make_dataloder(train_ds, valid_ds)
    
    wav_to_logmel = Wav2Logmel(device=device)
    # Clear the session and build the model
    model = BirdCLEF23Net(num_classes=CFG.num_classes)
    # Load birdclef pretrained weights
    if CFG.pretrain == True:
        weight_path = CFG.pretrained_model_path + f'/fold-{fold}.pth'
        weight=torch.load(weight_path)
        #weight=drop_weight(weight)
        print('load pre-trained model : ', weight_path)
        print(model.load_state_dict(weight, strict=False))
    model.to(device)
    wav_to_logmel.to(device)
    
    print('#' * 25)
    print('#### Training')
    print('#### Fold: %i | Image Size: (%i, %i) | Model: %s | Batch Size: %i | Scheduler: %s' %
        (fold + 1, *CFG.img_size, CFG.model_name, CFG.batch_size, CFG.scheduler))
    print('#### Num Train: {:,} | Num Valid: {:,}'.format(len(train_paths), len(valid_paths)))

    optimizer = com.get_optimizer(model)
    # TODO com.get_scheduler
    scheduler = CosineLRScheduler(optimizer, t_initial=CFG.epochs, lr_min=CFG.lr_min, 
                                  warmup_t=CFG.warmup_t, warmup_lr_init=CFG.warmup_lr_init, warmup_prefix=True)

    best_score = -1
    best_epoch = -1

    for epoch in range(2 if CFG.debug else CFG.epochs):
        # Training
        model.train()
        epoch_loss = 0
        for batch_idx, (inputs, sample_info) in enumerate(tqdm(train_dataloader)):
            inputs, targets = inputs.to(device), sample_info['multi_label_target'].to(device) #sample_info['target'].to(device)
            #targets = F.one_hot(targets, num_classes=CFG.num_classes).float()
            
            optimizer.zero_grad()
            logmel = wav_to_logmel(inputs)
            with torch.cuda.amp.autocast():
                outputs, mix_targets = model(logmel, targets)
                loss = modeler.loss_fn(outputs, mix_targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            if np.isinf(loss.item()) or np.isnan(loss.item()):
                print(f'Bad loss, skipping the batch {batch_idx}')
                del loss, outputs, mix_targets
                gc_collect()
                continue
            epoch_loss += loss.item()
            # wandb logger (Train loss)
            if CFG.wandb:
                run.log({'loss': loss.item()})
        scheduler.step(epoch+1)

        # Validation
        model.eval()
        val_loss = 0
        val_preds = []
        val_true = []
        with torch.no_grad():
            for inputs, sample_info in tqdm(val_dataloader):
                #inputs, targets = inputs.to(device), sample_info['multi_label_target'].to(device)
                inputs, targets = inputs.to(device), sample_info['target'].to(device)
                targets = F.one_hot(targets, num_classes=CFG.num_classes).float()
                
                logmel = wav_to_logmel(inputs)
                outputs = model(logmel)
                loss = modeler.loss_fn(outputs, targets)
                outputs = outputs["logit"]
                outputs = torch.sigmoid(outputs)
                #outputs = torch.softmax(outputs, dim=1)
                val_loss += loss.item()
                val_preds.append(outputs.detach().cpu().numpy())
                val_true.append(targets.detach().cpu().numpy())

        val_preds = np.vstack(val_preds)
        val_true = np.vstack(val_true)
        # Metrics
        val_score = com.padded_cmap(val_true, val_preds)
        # Checkpoint
        if val_score > best_score:
            best_score = val_score
            best_epoch = epoch + 1
            torch.save(model.state_dict(), f'fold-{fold}.pth')
            if CFG.wandb:
                art = wandb.Artifact("birdclef-2023", type="model")
                art.add_file(f'fold-{fold}.pth')
                run.log_artifact(art)

        print(f'Epoch: {epoch + 1} | Train Loss: {epoch_loss / len(train_dataloader)} | '
            f'Val Loss: {val_loss / len(val_dataloader)} | Val Padded_cmAP : {val_score}')
        
        # wandb logger
        lr = scheduler.get_epoch_values(epoch)[0]
        if CFG.wandb:
            run.log({'train_loss': epoch_loss / len(train_dataloader),
                    'lr': lr,
                    'epoch': epoch+1,
                    'valid_loss': val_loss / len(val_dataloader),
                    'valid_padded_cmAP': val_score,})
        
        
    # Load best checkpoint
    print('# Loading best model')
    model.load_state_dict(torch.load(f'fold-{fold}.pth'), strict=False)

    # Predict on the validation data for oof result
    print('# Infering OOF')
    model.eval()
    oof_pred_ = []
    with torch.no_grad():
        for inputs, sample_info in tqdm(val_dataloader):
            #inputs, targets = inputs.to(device), sample_info['multi_label_target'].to(device)
            inputs, targets = inputs.to(device), sample_info['target'].to(device)
            
            logmel = wav_to_logmel(inputs)
            outputs = model(logmel)
            outputs = outputs["logit"]
            outputs = torch.sigmoid(outputs)
            oof_pred_.append(outputs.detach().cpu().numpy())
            #print(oof_pred_.shape)

    oof_pred_ = np.concatenate(oof_pred_, axis=0)
    # oof_pred : 5
    oof_pred.append(oof_pred_)

    # Get ids and targets
    oof_true.append(valid_labels)
    oof_folds.append(np.ones_like(oof_true[-1], dtype='int8') * fold)
    oof_ids.append(valid_paths)

    # Save valid data prediction
    y_true = np.array(oof_true[-1])
    y_pred = np.argmax(oof_pred[-1], axis=-1)
    
    valid_df['pred'] = y_pred
    valid_df['miss'] = y_true != y_pred
    valid_df[CFG.class_names] = oof_pred[-1].tolist()
    # Log the metrics
    scores = {}
    cmAP = com.padded_cmap(com.one_hot_encode(y_true), oof_pred[-1])
    oof_val.append(best_score)
    print('\n>>> FOLD %i Primary_Padded_cmAP = %.3f' % (fold+1, cmAP))
    scores.update({'epoch': best_epoch,
                   'primary_cmAP': cmAP,})
    # wandb logger 
    if CFG.wandb:
        run.log(scores)
    # Show training plot
    # if CFG.training_plot:
    #     plot_history(history)
    # Log metrics, media to wandb
    if CFG.wandb:
        print('# WandB')
        log_wandb(valid_df)
        wandb.run.finish()
        #display(ipd.IFrame(run.url, width=1080, height=720))